# DL in production

In this notebook we will be exploring what using DL in a production pipeline or application might look like. To do this we will create a simple flask application that we deploy locally and submit jobs to. As in the last notebook we will by running a model on a clean field level NDVI timeseries. The hypothetical "user" of our application will submit a geometry and unique id in a request and receive the model output back. Serving a model on demand like this may make more sense than using Tasks in a scenario where you don't want to re-run large deployments often.

We will use the following DL API's in this exercise (in the flask app itself):
- [Scenes](https://docs.descarteslabs.com/descarteslabs/scenes/readme.html) - Query for and access imagery over our AOI
- [Storage](https://docs.descarteslabs.com/descarteslabs/client/services/storage/readme.html) - Store our model on the DL backend/cloud data store

We will use the following external Python packages:
- [requests](https://docs.python-requests.org/en/latest/) - Submit jobs to our flask app
- [geopandas](https://geopandas.org/en/stable/docs.html) - Import, transform, and query our reference dataset for Iowa agricultural fields
- [shapely](https://shapely.readthedocs.io/en/stable/) - Convert geometries into JSON format

In [ ]:
%pip install Flask

In [ ]:
import requests
import geopandas as gpd
import shapely.geometry as sg

We write a simple flask app that takes a request, accesses the provided geometry, pulls the clean timeseries using `get_ndvi_tseries`, pulls the model using Storage, loads the model, then returns the results of the model back to the "user".

```python
from flask import Flask, request
from .utils import get_ndvi_tseries
import descarteslabs as dl
import os.path as osp
from joblib import load

app = Flask(__name__)

@app.route("/getclass", methods=["POST", "GET"])
def get_field_class():
    content = request.json
    print(f"Recieved request: {content}")
    
    geom = content["geometry"]
    if "fid" in content.keys():
        fid = content["fid"]
    else:
        fid = ""
    
    print("Loading classifier")
    if not osp.exists("../models/classifier.joblib"):
        print("Classifier not found locally. Pulling from dl.Storage")
        dl.storage.get_file("classifier.joblib", "../models/classifier.joblib")
        
    clf = load("../models/classifier.joblib")
    
    print("Retrieving timeseries")
    ndvi_ts, ndvi_dates = get_ndvi_tseries(geom)
    
    result = {
        "class": clf.predict(ndvi_ts.reshape(1,-1))[0], 
        "fid": fid
    }
    
    return result
```

We can run the flask app "locally" by going to the bash shell and executing the following:
```bash
cd {path_to_notebooks}/dl-ea-notebooks/notebooks
export FLASK_APP=dl_flask_app
flask run
```

Once we execute the code above we should be able to see the following, indicating that the application is running.

![flask_app_running](../images/flask_app_running.png)

We will then submit requests to this app from this notebook using the `requests` library. Again we start by loading in our reference dataset and converting it to EPSG 4326.

In [ ]:
ia_fields = gpd.read_file("../data/IowaFieldBoundaries2019.shp")

In [ ]:
ia_fields = ia_fields.to_crs("EPSG:4326")

We choose a single field from the dataset then get it's geometry and unique identifier.

In [ ]:
# test_idx = 3000
# test_idx = 2000
test_idx = 100500

In [ ]:
ia_fields.iloc[test_idx]

In [ ]:
ia_fields.iloc[test_idx].geometry

In [ ]:
test_geom = sg.mapping(ia_fields.iloc[test_idx].geometry)
test_fid = ia_fields.iloc[test_idx].FBndID

We construct a request with the above geometry and id with the following format:

In [ ]:
request_json = {
    "geometry": test_geom,
    "fid": test_fid
}

In [ ]:
request_json

Finally we can submit the request to our local application and wait for a result! We should be able to monitor the status of the request being processed by watching the terminal where our flask application is running.

In [ ]:
res = requests.post('http://127.0.0.1:5000/getclass', json=request_json)

In [ ]:
res.json()

This flask application is quite simple and should only be run locally. Do not expose your local machine or credentials in a public flask application.

## What's next?

In this notebook series we've covered accessing raster data, uploading data to Catalog, deploying models in Tasks, and running a simple local application using DL as a remote sensing backend. For more examples, guides, and full documentation please see the docs [here](https://docs.descarteslabs.com/)! For any issues or questions please submit a ticket to the [DL customer support desk](https://descarteslabs.atlassian.net/servicedesk/customer/portals).